# Bulk CSV Geocode
*A notebook to [geocode](https://developers.arcgis.com/features/geocoding/) rows in a `csv`*

In [ ]:
# common imports
import csv
from arcgis.gis import GIS
from arcgis.geocoding import batch_geocode

## User Input
* **GIS**: Your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/). Not specifying a password creates a password prompt

In [ ]:
gis = GIS(username="mpayson_startups")

* **csv_path**: path to the `csv` to be geocoded
* **out_path**: path to the output `csv`
* **address_fields**: map geocode request fields to csv fields, can also be a SingleLine field string--more [here](https://developers.arcgis.com/rest/geocode/api-reference/geocoding-geocode-addresses.htm) 

In [ ]:
csv_path = "NYC Inspection test.csv"
out_path = "Geocode Results.csv"
address_fields = {
    "Address": "ADDRESS",
    "City": "CITY",
    "Region": "STATE",
    "Postal": "ZIPCODE"
}

## Functions

In [ ]:
# translate csv fields to geocode request fields
def format_request_address(row, address_fields):
    if type(address_fields) == str:
        return row[address_fields]
    return {field: row[address_fields[field]] for field in address_fields}

# geocoders have a request limit, the world geocoding service has a 1000 item limit
def geocode_chunk(row, n=1000):
    for i in range(0, len(row), n):
        yield row[i:i + n]

# get output location dictionary from geocode result
def get_location(res):
    if res:
        return {
            "x": res['location']['x'],
            "y": res['location']['y'],
            "score": res['score']
        }
    return {}

## Execution

In [ ]:
# read the csv
with open(csv_path, 'r') as data_csv:
    rows = csv.DictReader(data_csv)
    data = [row for row in rows]

# chunk the data for request limits
data_chunks = list(geocode_chunk(data))

In [ ]:
output = []

# iterate through chunks, format the address requests, geocode, merge location results with chunk data
for chunk in data_chunks:
    adrs = [format_request_address(r, address_fields) for r in chunk]
    results = batch_geocode(addresses=adrs)
    chunk_out = [{**r, **get_location(results[i])} for i, r in enumerate(chunk)]
    output += chunk_out

In [ ]:
# write output
with open(out_path, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=output[0].keys())
    writer.writeheader()
    for row in output:
        writer.writerow(row)
